In [7]:
import pandas as pd
gkg_prueba = pd.read_csv('URLS-CLUSTER.csv', sep='\t')


In [8]:
gkg_prueba.head()

,SOURCEURLS
0,https://edition.cnn.com/2024/08/16/health/fda-...
1,https://www.castanetkamloops.net/news/Canada/3...
2,https://www.castanetkamloops.net/news/Canada/4...
3,https://www.castanetkamloops.net/news/BC/49780...


In [14]:
from newspaper import Article

def extract_article_content(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return {
            'title': article.title,
            'text': article.text
        }
    except Exception as e:
        return {
            'title': f"Error extracting title: {str(e)}",
            'text': f"Error extracting content: {str(e)}"
        }

results = gkg_prueba['SOURCEURLS'].apply(extract_article_content)

gkg_prueba['ARTICLE_TITLE'] = results.apply(lambda x: x['title'])

In [15]:
gkg_prueba

,SOURCEURLS,ARTICLE_CONTENT,ARTICLE_TITLE
0,https://edition.cnn.com/2024/08/16/health/fda-...,CNN —\n\nThe US Food and Drug Administration i...,FDA may greenlight updated Covid-19 vaccines a...
1,https://www.castanetkamloops.net/news/Canada/3...,Photo: The Canadian Press\n\nFor the second ti...,Second COVID vaccine shows early success in U....
2,https://www.castanetkamloops.net/news/Canada/4...,Photo: The Canadian Press A global technology ...,"Canadian flights, hospitals, border disrupted ..."
3,https://www.castanetkamloops.net/news/BC/49780...,Photo: Colin Dacre Kelowna General Hospital.\n...,B.C. hospitals pivot to paper amid CrowdStrike...


In [16]:
from dotenv import load_dotenv
import os
import openai
from openai import OpenAI

load_dotenv()
api_key = os.getenv("API_KEY")

client = OpenAI(api_key=api_key)


def generate_summary(text):
    if text.startswith("Error extracting content"):
        return "No summary available due to extraction error"
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18",  
            messages=[
                {"role": "system", "content": "You are a helpful assistant that summarizes articles."},
                {"role": "user", "content": f"Summarize the following article in a few sentences: {text[:4000]}"}
            ],
            max_tokens=150
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error generating summary: {str(e)}"
    
gkg_prueba2 = gkg_prueba.head(10)


gkg_prueba2['ARTICLE_CONTENT'] = gkg_prueba2['SOURCEURLS'].apply(extract_article_text)
gkg_prueba2['SUMMARY'] = gkg_prueba2['ARTICLE_CONTENT'].apply(generate_summary)


gkg_prueba2.to_csv('dataset_actualizado_.csv', index=False)
gkg_prueba2.head()

,SOURCEURLS,ARTICLE_CONTENT,ARTICLE_TITLE,SUMMARY
0,https://edition.cnn.com/2024/08/16/health/fda-...,CNN —\n\nThe US Food and Drug Administration i...,FDA may greenlight updated Covid-19 vaccines a...,The US FDA is expected to approve updated COVI...
1,https://www.castanetkamloops.net/news/Canada/3...,Photo: The Canadian Press\n\nFor the second ti...,Second COVID vaccine shows early success in U....,Moderna has announced that its COVID-19 vaccin...
2,https://www.castanetkamloops.net/news/Canada/4...,Photo: The Canadian Press A global technology ...,"Canadian flights, hospitals, border disrupted ...",A global technology outage caused significant ...
3,https://www.castanetkamloops.net/news/BC/49780...,Photo: Colin Dacre Kelowna General Hospital.\n...,B.C. hospitals pivot to paper amid CrowdStrike...,A global technology outage caused by a faulty ...


In [17]:
from IPython.display import display

# Mostrar todas las filas y columnas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

display(gkg_prueba2[['ARTICLE_CONTENT', 'SUMMARY']])

,ARTICLE_CONTENT,SUMMARY
0,"CNN —\n\nThe US Food and Drug Administration is poised to sign off as soon as this week on updated Covid-19 vaccines targeting more recently circulating strains of the virus, according to two sources familiar with the matter, as the country experiences its largest summer wave in two years.\n\nThe agency is expected to greenlight updated mRNA vaccines from Moderna and Pfizer/BioNTech that target a strain of the virus called KP.2, said the sources, who declined to be named because the timing information isn’t public. It was unclear whether the agency simultaneously would authorize Novavax’s updated shot, which targets the JN.1 strain.\n\nThe move would be several weeks ahead of last year’s version of the vaccine, which got FDA signoff on September 11.\n\n“Now is the time to get a dose with this surge,” Dr. Michael Osterholm, director of the Center for Infectious Disease Research and Policy at the University of Minnesota, told CNN.\n\nOsterholm said on his podcast last week that he recently got a dose of last season’s vaccine in order to increase his immunity while the virus is circulating at such high levels and amid uncertainty around when new shots would become available.\n\nHe added that he’ll now wait to get the updated one in four months, the interval recommended by health officials.\n\nIn June, the US Centers for Disease Control and Prevention recommended that everyone over 6 months old receive both an updated Covid-19 vaccine and a flu shot this year.\n\nRepresentatives for Pfizer and Moderna told CNN that the companies had ample supply of their updated Covid vaccines and would be ready to ship doses upon approval. Moderna’s spokesman said it expects the vaccine to be available in stores within days of FDA signoff.\n\nNovavax’s vaccine is based on protein technology, which takes longer to manufacture than mRNA vaccines. The company’s executives told investors on a conference call last week that it anticipated that its updated vaccine would be arriving in warehouses this month and that it’s expected to be ready for distribution when authorized. A spokesperson for Novavax didn’t immediately respond to a request for comment Friday.\n\nA spokesperson for the FDA said the agency can’t comment on timing of product applications but noted that it “anticipates taking timely action to authorize or approve updated COVID-19 vaccines in order to make vaccines available this fall.”\n\nGet CNN Health's weekly newsletter Sign up here to get The Results Are In with Dr. Sanjay Gupta every Tuesday from the CNN Health team.\n\nLevels of the SARS-CoV-2 virus, which causes Covid-19, measured in wastewater are at “very high” levels nationally, according to CDC data, sparking the highest summer peak in the US since July 2022. Monitoring of viral levels in wastewater can give a picture of how widespread the virus is as testing and other forms of monitoring the virus have fallen off.\n\nMeasures of severe disease, including rates of hospitalization and death, have been rising, according to the CDC, but they’re nowhere near levels seen in previous years.\n\nWaves of the virus are driven by both waning immunity and new variants, experts say. The prevalent strain in the US now is KP.3.1.1, according to CDC data, estimated to account for 37% of cases over the past two weeks. That’s triple its level a month ago.\n\nKP.3.1.1 and KP.2 – the strain included in the updated mRNA vaccines – are both offshoots of JN.1, the target of Novavax’s shot, and all are versions of the Omicron variant.\n\nCNN’s Brenda Goodman contributed to this report.","The US FDA is expected to approve updated COVID-19 vaccines from Moderna and Pfizer/BioNTech targeting the KP.2 strain, coinciding with a significant summer wave of infections. Approval may come this week, ahead of last year’s timeline. Experts emphasize the importance of vaccination amid rising cases; the CDC recommends that individuals over six months receive both an updated COVID-19 vacc

In [18]:
import numpy as np
def get_embedding(text,model="text-embedding-3-small"):
    text = text.replace("\n", " ")  
    return client.embeddings.create(input=[text], model=model).data[0].embedding

gkg_prueba2['EMBEDDING'] = gkg_prueba2['SUMMARY'].apply(lambda x: get_embedding(x,model="text-embedding-3-small"))
embedding_list = np.array(gkg_prueba2['EMBEDDING'].tolist())
embedding_list.shape  

(4, 1536)

In [20]:
gkg_prueba2
gkg_prueba2.to_csv('prueba_cluster.csv', index=False)

In [21]:
gkg_prueba2.head()

,SOURCEURLS,ARTICLE_CONTENT,ARTICLE_TITLE,SUMMARY,EMBEDDING
0,https://edition.cnn.com/2024/08/16/health/fda-updated-covid-19-vaccines/index.html,"CNN —\n\nThe US Food and Drug Administration is poised to sign off as soon as this week on updated Covid-19 vaccines targeting more recently circulating strains of the virus, according to two sources familiar with the matter, as the country experiences its largest summer wave in two years.\n\nThe agency is expected to greenlight updated mRNA vaccines from Moderna and Pfizer/BioNTech that target a strain of the virus called KP.2, said the sources, who declined to be named because the timing information isn’t public. It was unclear whether the agency simultaneously would authorize Novavax’s updated shot, which targets the JN.1 strain.\n\nThe move would be several weeks ahead of last year’s version of the vaccine, which got FDA signoff on September 11.\n\n“Now is the time to get a dose with this surge,” Dr. Michael Osterholm, director of the Center for Infectious Disease Research and Policy at the University of Minnesota, told CNN.\n\nOsterholm said on his podcast last week that he recently got a dose of last season’s vaccine in order to increase his immunity while the virus is circulating at such high levels and amid uncertainty around when new shots would become available.\n\nHe added that he’ll now wait to get the updated one in four months, the interval recommended by health officials.\n\nIn June, the US Centers for Disease Control and Prevention recommended that everyone over 6 months old receive both an updated Covid-19 vaccine and a flu shot this year.\n\nRepresentatives for Pfizer and Moderna told CNN that the companies had ample supply of their updated Covid vaccines and would be ready to ship doses upon approval. Moderna’s spokesman said it expects the vaccine to be available in stores within days of FDA signoff.\n\nNovavax’s vaccine is based on protein technology, which takes longer to manufacture than mRNA vaccines. The company’s executives told investors on a conference call last week that it anticipated that its updated vaccine would be arriving in warehouses this month and that it’s expected to be ready for distribution when authorized. A spokesperson for Novavax didn’t immediately respond to a request for comment Friday.\n\nA spokesperson for the FDA said the agency can’t comment on timing of product applications but noted that it “anticipates taking timely action to authorize or approve updated COVID-19 vaccines in order to make vaccines available this fall.”\n\nGet CNN Health's weekly newsletter Sign up here to get The Results Are In with Dr. Sanjay Gupta every Tuesday from the CNN Health team.\n\nLevels of the SARS-CoV-2 virus, which causes Covid-19, measured in wastewater are at “very high” levels nationally, according to CDC data, sparking the highest summer peak in the US since July 2022. Monitoring of viral levels in wastewater can give a picture of how widespread the virus is as testing and other forms of monitoring the virus have fallen off.\n\nMeasures of severe disease, including rates of hospitalization and death, have been rising, according to the CDC, but they’re nowhere near levels seen in previous years.\n\nWaves of the virus are driven by both waning immunity and new variants, experts say. The prevalent strain in the US now is KP.3.1.1, according to CDC data, estimated to account for 37% of cases over the past two weeks. That’s triple its level a month ago.\n\nKP.3.1.1 and KP.2 – the strain included in the updated mRNA vaccines – are both offshoots of JN.1, the target of Novavax’s shot, and all are versions of the Omicron variant.\n\nCNN’s Brenda Goodman contributed to this report.","FDA may greenlight updated Covid-19 vaccines as soon as this week, sources say","The US FDA is expected to approve updated COVID-19 vaccines from Moderna and Pfizer/BioNTech targeting the KP.2 strain, coinciding with a significant summer wave of infections. Approval may come

In [31]:
import pandas as pd
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import pairwise_distances

# Convertir embeddings a array numpy
X = np.array(gkg_prueba2['EMBEDDING'].tolist())

# Calcular matriz de distancias (1 - similitud coseno)
distance_matrix = pairwise_distances(X, metric='cosine')

# Realizar clustering jerárquico
n_clusters = 2  # Número de clusters deseado
clustering = AgglomerativeClustering(n_clusters=n_clusters, metric='precomputed', linkage='average')
clustering.fit(distance_matrix)

# Añadir etiquetas de cluster al DataFrame
gkg_prueba2['cluster'] = clustering.labels_

# Mostrar información de los clusters
for i in range(n_clusters):
    print(f"\nCluster {i+1}:")
    cluster_data = gkg_prueba2[gkg_prueba2['cluster'] == i]
    
    for _, row in cluster_data.iterrows():
        print(f"\nTítulo: {row['ARTICLE_TITLE']}")
        print(f"URL: {row['SOURCEURLS']}")
        print(f"Resumen: {row['SUMMARY']}")
        print("-" * 80)

# Análisis adicional
print("\nEstadísticas de los clusters:")
print(gkg_prueba2.groupby('cluster').agg({
    'ARTICLE_TITLE': 'count'
}).rename(columns={'ARTICLE_TITLE': 'count'}))

print("\nPalabras clave por cluster:")
gkg_prueba2['keywords'] = gkg_prueba2['ARTICLE_TITLE'].str.split()
for i in range(n_clusters):
    keywords = gkg_prueba2[gkg_prueba2['cluster'] == i]['keywords'].sum()
    # Convertimos el set a una lista para poder indexar
    top_keywords = list(set(keywords))[:10]
    print(f"Cluster {i+1}: {', '.join(top_keywords)}")



Cluster 1:

Título: Canadian flights, hospitals, border disrupted during global technology outage - Canada News
URL: https://www.castanetkamloops.net/news/Canada/497777/Canadian-flights-hospitals-border-disrupted-during-global-technology-outage
Resumen: A global technology outage caused significant disruptions in Canada on Friday, grounding flights, impacting hospital services, and delaying border crossings. The issues were linked to Microsoft 365 services, although a cybersecurity firm clarified it was not a cyberattack. Airports like Toronto Pearson and Montréal-Trudeau experienced delays, with Porter Airlines canceling flights until noon ET, while Air Canada reported minimal impact. Some hospitals faced system disruptions, but critical operations continued. Microsoft announced efforts to resolve the issues and indicated improvements in service availability.
--------------------------------------------------------------------------------

Título: B.C. hospitals pivot to paper amid C